## Exemple de génération de texte avec un réseau récurrent
On reprend ici la tâche de génération de noms de famille pour illuster l'utilisation d'un réseau récurrent pour la génération de textes. Les données utilisées contiennent des noms dans 18 langues d'origine. À partir de ces exemples, on entraîne un seul modèle RNN pour toutes les langues.

### 1. Préparation du jeu de données

In [28]:
import json

train_filename = "./data/train_names.json"
test_filename = './data/test-names-t2.txt'

names_by_origin = {}  # un dictionnaire qui contient une liste de noms pour chaque langue d'origine
all_origins = []  # la liste des 18 langue d'origines de noms

BOS = "$"  # le caractère indiquant le début d'un nom
EOS = "!"  # le caractère indiquant la fin d'un nom

def load_names(input_file):
    with open(input_file, 'r') as names_file:
        names = json.load(names_file)
    origins = list(names.keys())
    return origins, names

def vocabulary():
    voc = set()
    for origin in all_origins:
        for name in names_by_origin[origin]:
            for letter in name:
                voc.add(letter)
    voc = list(voc)
    voc.sort()
    voc.append(BOS)
    voc.append(EOS)
    return voc

In [30]:
all_origins, names_by_origin = load_names(train_filename)
all_letters = vocabulary()
all_origins

## 2. Transformation des inputs du réseau

On prépare une classe NameDataset qui hérite de la classe Dataset et qui est utilisée par un Dataloader de PyTorch pour gérer les données durant l'entraînement du modèle.

In [31]:
all_originsfrom torch.utils.data import Dataset

class NameDataset(Dataset):

    def __init__(self, names_by_origin, origin_list, vocabulary, bos_tag, eos_tag):
        self.vocabulary = vocabulary
        self.origin_list = origin_list
        self.tokenized_names = []
        self.origin_index = []
        self._generate_input_pairs(names_by_origin, bos_tag, eos_tag)

    def __len__(self):
        return len(self.tokenized_names)

    def __getitem__(self, item):
        return (torch.LongTensor([self.origin_index[item]]), torch.LongTensor(self.tokenized_names[item][:-1])), torch.LongTensor(self.tokenized_names[item][1:])

    def _generate_input_pairs(self, names_by_origin, bos_tag, eos_tag):
        bos_index = self.vocabulary.index(bos_tag)
        eos_index = self.vocabulary.index(eos_tag)
        for origin in names_by_origin:
            for name in names_by_origin[origin]:
                name_as_index_list = [bos_index] + [self.vocabulary.index(letter.lower()) for letter in name] + [eos_index]
                self.tokenized_names.append(name_as_index_list)
                self.origin_index.append(self.origin_list.index(origin))

[]

In [ ]:
test_names_by_origin = json.load(open(test_filename,'r'))
name_dataset = NameDataset(names_by_origin, all_origins, all_letters, BOS, EOS)
test_name_dataset = NameDataset(test_names_by_origin, all_origins, all_letters, BOS, EOS)

 La fonction generate_one_hot_vector_table_for_classes crée un one-hot vector pour chaque origine d'un nom. Ainsi la table qui est généré contiendra 18 vecteurs, chacun ayant un 1 à la position correspondant à l'orgine, le reste état des 0.

On utilise la même fonction pour convertir les lettres du vocabulaire en one-hot vector.

In [ ]:
import torch
import numpy as np

def generate_one_hot_vector_table_for_classes(classes):
    nb_class = len(classes)
    one_hot_vectors = np.zeros((nb_class, nb_class))
    for i in range(nb_class):
        one_hot_vectors[i,i] = 1
    return one_hot_vectors

# On transforme les matrice numpy en tensor pour les insérer directement dans l'architecture neuronale
letter_vectors = torch.FloatTensor(generate_one_hot_vector_table_for_classes(all_letters))
origin_vectors = torch.FloatTensor(generate_one_hot_vector_table_for_classes(all_origins))

## 3. Création d'une architecture de réseau récurrent
Le modèle GenerationRNN est un réseau récurrent qui prend en entrée la lettre précédente du nom à générer et la langue d'origine. Chacune est représentée par un one-hot vector. Donc, l'input du réseau est la concaténation des one-hot vectors de la langue d'origine et de la lettre. En sortie, le réseau estime un score pour chacune des lettre du vocabulaire.

Dans cet exemple, les cellules de la couche caché du réseau récurrent sont des GRUs (voir le matériel du cours pour plus de détails).

L'état cachée produite par le GRU est repris par un couche linéaire pour choisir la prochaine lettre à générer.

Note: le terme embeddings est utilisé ici plutôt librement, car le réseau ne construit pas explicitement de plongements de caractères. Les one-hot vecteurs sont connectés directement à la couche cachée. Une couche cachée additionnelle de projection serait nécessaire pour obtenir des embeddings suite à l'entraînement du réseau.

In [ ]:
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence


class GenerationRNN(nn.Module):

    def __init__(self, vector_1, vector_2, hidden_state_size, nb_classes):
        super().__init__()
        self.class_embeddings = nn.Embedding.from_pretrained(vector_1)
        self.token_sequence_embeddings = nn.Embedding.from_pretrained(vector_2)
        joined_embedding_size = self.class_embeddings.embedding_dim + self.token_sequence_embeddings.embedding_dim
        self.rnn = nn.GRU(joined_embedding_size, hidden_state_size, 2, bidirectional=False)
        self.classification_layer = nn.Linear(hidden_state_size, nb_classes)

    def forward(self, index_1, index_2, sequence_length):
        joined_embedding = self._join_embedding(index_1, index_2, sequence_length)
        rnn_output = self._handle_rnn_output(joined_embedding, sequence_length)
        output = self.classification_layer(rnn_output)
        return output, sequence_length

    def _join_embedding(self, index_1, index_2, sequence_length):

        # On a une origin pour chacun des noms de la batch
        # Dimensions du tensor: 1 x origin_size x b 
        class_embedding = self.class_embeddings(index_1) 
        
        # On a au maximum n_lettres pour tous les noms de la batch 
        # Dimensions du tensor: n_lettres x vocabulary_size x b
        sequence_embeddings = self.token_sequence_embeddings(index_2) 
        
        # on crée une "tuile" du vecteur de l'origine en le copiant pour chaque lettre du nom 
        # Dimensions du tensor: n_lettres x origine_size x b
        max_sequence_length = torch.max(sequence_length)
        tiled_class_embedding = class_embedding.repeat(1, max_sequence_length, 1)  
        
        # On colle la tuile au dessus des vecteurs de lettres du nom: 
        # Dimensions du tensor: n_lettres x (origin_size + vocabulary_size) x b
        joined_embedding = torch.cat((tiled_class_embedding, sequence_embeddings), dim=2) 

        return joined_embedding

    def _handle_rnn_output(self, x, x_lenghts):
        # On "pack" les batch pour les envoyer dans le RNN
        packed_batch = pack_padded_sequence(x, x_lenghts, batch_first=True, enforce_sorted=False)

        # On s'intéresse cette fois-ci aux output après chaque mot
        rnn_output, _ = self.rnn(packed_batch)

        # On "repad" les output pour les remettre dans une forme utilisable
        unpacked_rnn_output, _ = pad_packed_sequence(rnn_output, batch_first=True)

        return unpacked_rnn_output


## 4. Création d'un métrique de perte (loss function)
La métrique de perte est l'entropie croisée qui correspond ici à la classification correcte de la prochaine lettre sachant toutes les lettres précédentes.

In [ ]:
class HiddenStateClassificationLoss(nn.Module):

    def __init__(self):
        super().__init__()

    def forward(self, model_output, y_truth):
        y_pred, sequence_length = model_output # y_pred : batch x name length x nb letters
        loss = torch.FloatTensor([0])
        # Pour chaque exemple d'entraînement
        for example_index in range(y_pred.size()[0]):
            example_length = sequence_length[example_index]
            outputs_to_predict = y_pred[example_index, 0:example_length, :]
            true_value = y_truth[example_index][0:example_length]
            # On compare simultanément les prédictions de toutes les lettres du nom
            loss += nn.functional.cross_entropy(outputs_to_predict, true_value)
        return loss


## 5. Entraînement du modèle de réseau récurrent¶


In [ ]:
def pad_batch(batch):
    x = [x for x, y in batch]
    x_class = torch.stack([origin for origin, _ in x], dim=0)
    x_sequence = [sequence for _, sequence in x]
    x_true_length = [len(x) for x in x_sequence]
    y = [y for x, y in batch]
    return ((x_class, pad_sequence(x_sequence, batch_first=True), torch.LongTensor(x_true_length)), pad_sequence(y, batch_first=True))

from torch.utils.data import DataLoader

train_dataloader = DataLoader(name_dataset, batch_size=16, shuffle=True, collate_fn=pad_batch)
test_dataloader = DataLoader(test_name_dataset, batch_size=16, shuffle=True, collate_fn=pad_batch)


In [ ]:
from poutyne.framework import Experiment
from poutyne import set_seeds
set_seeds(42)

model = GenerationRNN(origin_vectors, letter_vectors, 300, len(all_letters))

experiment = Experiment('model/rnn_generation', model, loss_function=HiddenStateClassificationLoss(), optimizer="adam")
experiment.train(train_dataloader, test_dataloader, epochs=30)


## 6. Génération de noms avec le modèle de réseau récurrent


In [ ]:
def convert_to_input(origin, current_name):
    origin_index = all_origins.index(origin)
    letter_indexes = [all_letters.index(letter) for letter in current_name]
    current_name_length = len(current_name)
    return torch.LongTensor([origin_index]), torch.LongTensor(letter_indexes).unsqueeze(0), torch.LongTensor([current_name_length])

def generate_name(language, starting_letter):
    current_name = "${}".format(starting_letter.lower())
    current_input = convert_to_input(language, current_name)
    next_letter = None
    while next_letter != EOS:
        output = model(*current_input)[0][:,-1,:].detach().numpy()
        best = np.argmax(output)
        next_letter = all_letters[best]
        current_name += next_letter
        current_input = convert_to_input(language, current_name)
    return current_name[1:-1]


In [ ]:
import string
import random

def generate_names_for_each_origin(nb_generations):
    for origin in all_origins:
        generated_names = []
        letters = random.sample(string.ascii_lowercase, nb_generations)
        for letter in letters:
            first_letter = letter.upper()
            new_name = generate_name(origin, first_letter).capitalize()
            generated_names.append(new_name)
        print(origin, generated_names)

generate_names_for_each_origin(5)
